# Install conda on your Colab environment

Ignore this first cell if you are running the notebook in a local environment.

One can still run it locally but it will have no effect.

In [1]:
# Run this cell first - it will install a conda distribution (mamba)
# on your Drive then restart the kernel automatically 
# (don't worry about the crashing/restarting kernel messages)
# It HAS to be runned FIRST everytime you use the notebook in colab

import os
import sys
RunningInCOLAB  = 'google.colab' in str(get_ipython())

if RunningInCOLAB:
    !pip install -q condacolab
    import condacolab
    condacolab.install()

# Set up your Colab or local environment
# Then import libraries

Run this cell in both cases of use (local or Colab)

In [4]:
import os
import sys
RunningInCOLAB  = 'google.colab' in str(get_ipython())

if RunningInCOLAB:
    
    # Check everything is fine with conda in Colab
    import condacolab
    condacolab.check()
    
    # Mount your drive environment in the colab runtime
    from google.colab import drive
    drive.mount('/content/drive',force_remount=True)
    
    # Change this variable to your path on Google Drive to which the repo has been cloned
    # If you followed the colab notebook 'repo_cloning.ipynb', nothing to change here
    repo_path_in_drive = '/content/drive/My Drive/Github/amn_release/'
    # Change directory to your repo cloned in your drive
    DIRECTORY = repo_path_in_drive
    os.chdir(repo_path_in_drive)
    # Copy the environment given in the environment_amn_light.yml
    !mamba env update -n base -f environment_amn_light.yml
    
    # This is one of the few Colab-compatible font
    font = 'Liberation Sans'
    
else:
    
    # In this case the local root of the repo is our working directory
    DIRECTORY = './'
    font = 'arial'

# printing the working directory files. One can check you see the same folders and files as in the git webpage.
print(os.listdir(DIRECTORY))

from Build_Model import *

# We declare this function here and not in the
# function-storing python file to modify it easily
# as it can change the printouts of the methods
def printout(filename, Stats, model, time): 
    # printing Stats
    print('Stats for %s CPU-time %.4f' % (filename, time))
    print('R2 = %.4f (+/- %.4f) Constraint = %.4f (+/- %.4f)' % \
          (Stats.train_objective[0], Stats.train_objective[1],
           Stats.train_loss[0], Stats.train_loss[1]))
    print('Q2 = %.4f (+/- %.4f) Constraint = %.4f (+/- %.4f)' % \
          (Stats.test_objective[0], Stats.test_objective[1],
           Stats.test_loss[0], Stats.test_loss[1]))

['README.md', 'Duplicate_Model.ipynb', 'Build_Model_Dense.ipynb', 'Build_Dataset.py', 'Dataset_experimental', '.ipynb_checkpoints', '.git', 'Build_Experimental.ipynb', 'Reservoir', 'Dataset_model', 'Figures.ipynb', 'Result', 'Figures', '.gitignore', 'Duplicate_Model.py', 'LICENSE', 'Build_Dataset.ipynb', 'Dataset_input', '__pycache__', 'Build_Experimental.py', 'old', 'environment_amn.yml', 'Build_Model.py', 'Build_Model.ipynb', '.DS_Store']


## (4) Reservoir computing with experimental data set

In [6]:
# Create, train and evaluate RC models on experimental training set 
# Repeat the process with different seeds
# This cell takes several hours to execute

seed = 10
Maxloop, Q2, PRED = 3, [], []  
for Nloop in range(Maxloop):
    # What you can change 
    seed = Nloop
    np.random.seed(seed=seed)  
    trainname = 'iML1515_EXP'
    reservoirname = 'iML1515_UB_AMN_QP'
    # End of What you can change
  
    # Create model 
    trainingfile = DIRECTORY+'Dataset_input/'+trainname
    reservoirfile = DIRECTORY+'Reservoir/'+reservoirname
    X, Y = read_XY(trainingfile, 38)
    model = RC_Model(reservoirfile = reservoirfile, X=X, Y=Y, 
                n_hidden_prior = 1, hidden_dim_prior = 500, activation_prior = 'relu',
                epochs = 1000, train_rate = 1.0e-4, xfold = 0,
                verbose=True) 
    model.printout()

    # Train and evaluate
    start_time = time.time()
    reservoir, pred, stats, _ = train_evaluate_model(model, verbose=True)
    delta_time = time.time() - start_time

    # Printing cross-validation results
    printout(reservoirname, stats, model, delta_time)
    r2 = r2_score(model.Y, pred[:,0], multioutput='variance_weighted')
    print('Iter', Nloop, 'Collated Q2', r2)
    Q2.append(r2)
    PRED.append(pred)

# Some printings and savings
Q2, PRED = np.asarray(Q2), np.asarray(PRED)
print(PRED.shape)
obj = PRED[:,:,0]
print('Averaged Q2 = %.4f (+/- %.4f)' % (np.mean(Q2), np.std(Q2)))
filename = DIRECTORY+'Result/'+reservoirname+'_'\
+str(model.model_type)+'_Q2.csv'
np.savetxt(filename, Q2, delimiter=',')
filename = DIRECTORY+'Result/'+reservoirname+'_'\
+str(model.model_type)+'_PRED.csv'
np.savetxt(filename, obj, delimiter=',')

# Saving input / output of the first prediction for Cobra runs
pred, obj = PRED[0], pred[:,0]
Loss_SV, Loss_Vin, Loss_Vpos = pred[:,1], pred[:,2], pred[:,3]
print('Loss SV average %.4f max %.4f' % (np.mean(Loss_SV), np.max(Loss_SV)))
print('Loss Vin average %.4f max %.4f' % (np.mean(Loss_Vin), np.max(Loss_Vin)))
print('Loss Vpos average %.4f max %.4f' % (np.mean(Loss_Vpos), np.max(Loss_Vpos)))
V = pred[:, 4:4+model.S.shape[1]]
Vin = pred[:, 4+model.S.shape[1]:]
Vin = model.res.scaler * Vin # rescale back for Cobra
XY = np.concatenate((Vin, Y), axis=1)
filename = DIRECTORY+'Result/'+reservoirname+'_'\
+str(model.model_type)+'_solution_for_Cobra.csv'
np.savetxt(filename, XY, delimiter=',')

number of reactions:  550 550
number of metabolites:  1083
filtered measurements size:  1
RC reservoir file: ./Reservoir/iML1515_UB_AMN_QP
RC model type: RC_AMN
RC scaler: 0.0
RC model input dim: 38
RC model output dim: 1
RC model medium bound: UB
training set size (110, 38) (110, 1)
reservoir S, Pin, Pout matrices (1083, 550) (38, 550) (1, 550)
RC training epochs: 1000
RC training regression: True
RC training learn rate: 0.0001
RC training dropout: 0.25
RC training batch size: 5
RC training validation iter: 0
RC training xfold: 0
RC training early stopping: False
--------prior network --------
training file: None
model type: ANN_Dense
model scaler: 0.0
model input dim: 10
model output dim: 10
model medium bound: 
timestep: 0
no training set provided
nbr hidden layer: 1
hidden layer size: 500
activation function: relu
--------reservoir network-----
training file: ./Dataset_model/iML1515_UB
model type: AMN_QP
model scaler: 11.0
model input dim: 38
model output dim: 1104
model medium bou